In [ ]:
import requests
import numpy as np
import matplotlib.mlab as mlab
import matplotlib.pyplot as plt
import collections
import pandas as pd
from bs4 import BeautifulSoup

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats

from textblob import TextBlob

from typing import Union, List, Set
from tokenizer import tokenize

from typing import Dict
from collections import Counter
import math

In [ ]:
def gender(comments):
    m = 0
    f = 0
    for c in comments:
        if 'He ' in c or ' he ' in c or 'His ' in c or ' his ' in c or 'guy' in c or 'dude' in c or 'him' in c:
            m += 1
        if 'She ' in c or ' she ' in c or 'Her ' in c or ' her ' in c or 'lady' in c:
            f += 1
    
    gen = []
    if m > f:
        gen = ['M']
    elif m < f:
        gen = ['F']
    else:
        gen = ['N/A']
    
    return gen

def instructorinfo(ID):
    url = 'https://www.ratemyprofessors.com/ShowRatings.jsp?tid='+ID
    r = requests.get(url)
    html = BeautifulSoup(r.text, 'html.parser')
    
    name = html.find('h1')
    firstlast = []
    professorName=[]
    university=[]
    
    college=html.find('a',{"class": "school"}).string.strip()
    university.append(college)
    
    for span in name.find_all('span'):
        del span['class']
        firstlast.append(span)
        
    professorName.append(firstlast[0].string.strip()+' '+firstlast[2].string.strip())
        
    return(professorName,university)

def parse(ID):
    l=[]
    for page in range(15):
        r_sanfran = requests.get("http://www.ratemyprofessors.com/paginate/professors/ratings?tid="+ID+"&", params={'page': page}).json()
        q=r_sanfran["ratings"]
        l.append(q)
        
    a=l
    
    date=[]
    classID=[]
    overall=[]
    helpfulness=[]
    easiness=[]
    clarity=[]
    quality=[]
    TakeAgain=[]
    teacherGrade=[]
    comment=[]
    tag=[]
    commentNum=[]
    
    for i in range(1,100):
        for k in range(100):
            try:
                date.append(a[i][k]['rDate'])
                classID.append(a[i][k]['rClass'])
                overall.append(a[i][k]['rOverall'])
                helpfulness.append(a[i][k]['rHelpful'])
                easiness.append(a[i][k]['rEasy'])
                clarity.append(a[i][k]['rClarity'])
                quality.append(a[i][k]['quality'])
                TakeAgain.append(a[i][k]['rWouldTakeAgain'])
                teacherGrade.append(a[i][k]['teacherGrade'])
                comment.append(a[i][k]['rComments'])
                tag.append(a[i][k]['teacherRatingTags'])
            
            except:
                pass
    
    namelist=instructorinfo(ID)[0]
    namelist=namelist*len(date)
    
    genderlist=gender(comment)*len(date)
    
    collegelist=instructorinfo(ID)[1]
    collegelist=collegelist*len(date)
    
    Num=int(len(date))
    commentNum.append(Num)
    commentNum=commentNum*len(date)
    
    pol = []
    subj = []
    for i in comment:
        sentence = TextBlob(i)
        pol.append(round(sentence.sentiment.polarity,2))
        subj.append(round(sentence.sentiment.subjectivity,2))
      
    df= pd.DataFrame(
        {'Name':namelist,
         'Gender':genderlist,
         'Date': date,
         'College':collegelist,
         'ClassID': classID,
         'OverallScore': overall,
         'Helpfulness':helpfulness,
         'Easiness':easiness,
         'Clarity':clarity,
         'Quality':quality,
         'TakeAgain':TakeAgain,
         'TeacherGrade':teacherGrade,
         'Tag':tag,
         'Polarity':pol,
         'Subjectivity':subj,
         'CommentNumber':commentNum,
         'Comment':comment
        })
    
    return(df)

In [ ]:
df=parse('2119096')
df

In [ ]:
ID = pd.read_csv('res/Professor ID.csv')
IDList = ID['ID'].tolist()

In [ ]:
for i in IDList:
    print(i)
    try:
        a=parse(str(i))
    except:
        pass
    
    df=df.append(a) 